In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [ ]:
# rawDF = spark.read.json("/mnt/data/y=2022/m=11/d=14/h=01/m=00/macAddress=000D3A40BE42/PT1H.json")
# rawDF = spark.read.json("/mnt/data/y=2022/m=11/d=14/h=01/m=00/*/PT1H.json")
# rawDF = spark.read.json("/mnt/data/y=2022/m=11/d=14/h=01/*/*/PT1H.json")
# rawDF = spark.read.json("/mnt/data/y=2022/m=11/d=14/*/*/*/PT1H.json")
rawDF = spark.read.json("/mnt/data/y=2022/m=11/*/*/*/*/PT1H.json")
# rawDF = spark.read.json("/mnt/data/y=2022/*/*/*/*/*/PT1H.json")
# rawDF = spark.read.json("dbfs:/mnt/data/logs/*/*.json", multiLine = "true")
# rawDF = spark.read.json("/mnt/data/logs/*.json", multiLine = "true")
# rawDF.show()

In [ ]:
recordsDF = rawDF.select(explode("records"))
recordsDF.printSchema()

In [ ]:
flows1DF = recordsDF.select(
    "col.time",
    "col.category",
    "col.macAddress",
    "col.properties.Version",
    explode("col.properties.flows"),
)
flows1DF.printSchema()
# flows1DF.show()

In [ ]:
flows2DF = flows1DF.select(
    "time", "category", "macAddress", "Version", "col.rule", explode("col.flows")
)
flows2DF.printSchema()
flows2DF.show(1, False)
# flows2DF.show()

In [ ]:
flowTuplesDF = flows2DF.select(
    "time",
    "category",
    "macAddress",
    "Version",
    "rule",
    "col.mac",
    explode("col.flowTuples"),
)
# flowTuplesDF.printSchema()
# flowTuplesDF.show(1,False)

finalDf = (
    flowTuplesDF.withColumn("epochtime", split(col("col"), ",").getItem(0))
    .withColumn("sourceip", split(col("col"), ",").getItem(1))
    .withColumn("destip", split(col("col"), ",").getItem(2))
    .withColumn("sourceport", split(col("col"), ",").getItem(3))
    .withColumn("destport", split(col("col"), ",").getItem(4))
    .withColumn("Protocol", split(col("col"), ",").getItem(5))
    .withColumn("TrafficFlow", split(col("col"), ",").getItem(6))
    .withColumn("TrafficDecision", split(col("col"), ",").getItem(7))
    .withColumn("FlowState", split(col("col"), ",").getItem(8))
    .withColumn("PacketsS2D", split(col("col"), ",").getItem(9))
    .withColumn("BytesSentS2D", split(col("col"), ",").getItem(10))
    .withColumn("PacketsD2S", split(col("col"), ",").getItem(11))
    .withColumn("BytesSentD2S", split(col("col"), ",").getItem(12))
    .drop("col")
)

finalDf.show(2, False)
finalDf.printSchema()

In [ ]:
print(f"Total processed flow tuple number: {finalDf.count()}")

### Save result to ADLS

In [ ]:
finalDf.write.format("com.databricks.spark.csv").mode("overwrite").save(
    "dbfs:/mnt/data/output/nsgflow_dbricks_merged.csv"
)